In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('Warehouse_and_Retail_Sales.csv')
print(data.head())

original_size = len(data)
print(original_size)

# Check for missing values in key columns
target_missing_values = data[['YEAR', 'MONTH', 'ITEM TYPE', 'RETAIL SALES', 'RETAIL TRANSFERS', 'WAREHOUSE SALES']].isnull().sum()
print("Missing Values Count:")
print(target_missing_values)

# Fill missing values with 0 and clean up column names
data.fillna(0, inplace=True)
data.columns = data.columns.str.replace(' ', '_')

# Check again to confirm no missing values remain
cleaned_values = data[['YEAR', 'MONTH', 'ITEM_TYPE', 'RETAIL_SALES', 'RETAIL_TRANSFERS', 'WAREHOUSE_SALES']].isnull().sum()
print("Missing Values Count after cleaned:")
print(cleaned_values)

# Filter data for relevant product categories: WINE, LIQUOR, BEER
alcohol_data = data[data['ITEM_TYPE'].isin(['WINE', 'LIQUOR', 'BEER'])]

# Create an SQLite database and save the filtered data into it
conn = sqlite3.connect("sales_data.db")
alcohol_data.to_sql("alcohol_sales", conn, if_exists="replace", index=False)

query = """
SELECT YEAR, MONTH, ITEM_TYPE, RETAIL_SALES, WAREHOUSE_SALES
FROM alcohol_sales
"""
integrated_data = pd.read_sql(query, conn)

integrated_data['Date'] = pd.to_datetime(integrated_data[['YEAR', 'MONTH']].assign(day=1))
integrated_data.set_index('Date', inplace=True)
conn.close()

# Split the data into separate datasets for each product type
wine_data = integrated_data[integrated_data['ITEM_TYPE'] == 'WINE']
liquor_data = integrated_data[integrated_data['ITEM_TYPE'] == 'LIQUOR']
beer_data = integrated_data[integrated_data['ITEM_TYPE'] == 'BEER']

# Function to prepare and process the data for modeling
def preprocess_data(data):
    # Aggregate sales data by month
    monthly_sales = data[['RETAIL_SALES', 'WAREHOUSE_SALES']].resample('MS').sum()
    monthly_sales['Month'] = monthly_sales.index.month

    # Add sine and cosine transformations for seasonal patterns
    monthly_sales['Month_sin'] = np.sin(2 * np.pi * monthly_sales['Month'] / 12)
    monthly_sales['Month_cos'] = np.cos(2 * np.pi * monthly_sales['Month'] / 12)

    # Create lagged features for past sales data
    for lag in range(1, 4):
        monthly_sales[f'Lag_{lag}_Retail'] = monthly_sales['RETAIL_SALES'].shift(lag)
        monthly_sales[f'Lag_{lag}_Warehouse'] = monthly_sales['WAREHOUSE_SALES'].shift(lag)

    # Remove rows with invalid or zero sales
    monthly_sales = monthly_sales[(monthly_sales['RETAIL_SALES'] > 0) & (monthly_sales['WAREHOUSE_SALES'] > 0)]
    
    # Fill missing values caused by lagging
    monthly_sales.fillna(0, inplace=True)
    return monthly_sales

# Function to train and evaluate the model
def train_and_evaluate(data, label):
    monthly_sales = preprocess_data(data)

    X = monthly_sales.drop(columns=['RETAIL_SALES', 'WAREHOUSE_SALES'])
    y_retail = monthly_sales['RETAIL_SALES']
    y_warehouse = monthly_sales['WAREHOUSE_SALES']

    # Standardize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    tscv = TimeSeriesSplit(n_splits=5)
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [3, 5, 10]}

    # Train the model for retail sales
    rf_model_retail = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=tscv)
    rf_model_retail.fit(X_scaled, y_retail)

    # Train the model for warehouse sales
    rf_model_warehouse = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=tscv)
    rf_model_warehouse.fit(X_scaled, y_warehouse)

    # Predict retail sales
    predictions_retail = rf_model_retail.predict(X_scaled)
    rmse_retail = np.sqrt(mean_squared_error(y_retail, predictions_retail))
    r2_retail = r2_score(y_retail, predictions_retail)
    mape_retail = mean_absolute_percentage_error(y_retail, predictions_retail)

    # Predict warehouse sales
    predictions_warehouse = rf_model_warehouse.predict(X_scaled)
    rmse_warehouse = np.sqrt(mean_squared_error(y_warehouse, predictions_warehouse))
    r2_warehouse = r2_score(y_warehouse, predictions_warehouse)
    mape_warehouse = mean_absolute_percentage_error(y_warehouse, predictions_warehouse)

    print(f"\n{label} Retail Sales Model Metrics:")
    print(f"RMSE (Retail): {rmse_retail:.2f}")
    print(f"R² (Retail): {r2_retail:.2f}")
    print(f"MAPE (Retail): {mape_retail:.2%}")

    print(f"\n{label} Warehouse Sales Model Metrics:")
    print(f"RMSE (Warehouse): {rmse_warehouse:.2f}")
    print(f"R² (Warehouse): {r2_warehouse:.2f}")
    print(f"MAPE (Warehouse): {mape_warehouse:.2%}")

    plt.figure(figsize=(12, 6))
    plt.plot(monthly_sales.index, y_retail, label='Actual Retail', linewidth=2)
    plt.plot(monthly_sales.index, predictions_retail, label='Predicted Retail', linestyle='--', color='orange')
    plt.title(f"{label} Retail Sales Prediction")
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12, 6))
    plt.plot(monthly_sales.index, y_warehouse, label='Actual Warehouse', linewidth=2)
    plt.plot(monthly_sales.index, predictions_warehouse, label='Predicted Warehouse', linestyle='--', color='green')
    plt.title(f"{label} Warehouse Sales Prediction")
    plt.xlabel('Date')
    plt.ylabel('Sales')
    plt.legend()
    plt.grid(True)
    plt.show()

train_and_evaluate(wine_data, 'WINE')
train_and_evaluate(liquor_data, 'LIQUOR')
train_and_evaluate(beer_data, 'BEER')
